## 用深层网络进行内容重建
效果很不好，根本看不清

### 构建计算图
修改：我们做风格转换不需要最后的fc，并且fc含有参数多，费时，所以我们这里注释掉

In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from PIL import Image #图像处理库
import time

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#VGG中自带常量，VGG有将图片进行预处理，其中一个步骤是normalization:
#减去image_net的RGB通道的各个均值
VGG_MEAN = [103.939, 116.779, 123.68] #在vggnet的code中

In [4]:
class VGGNet:
    """Builds VGG-16 net structure,
        load parameters from pre-trained models.
    """
    def __init__(self, data_dict):
        self.data_dict = data_dict
        
    def get_conv_filter(self, name): #抽取卷积参数
        """eg. conv1_1 = data_dict['conv1_1']"""
        #tf.constant() #因为模型是预处理好的，所以我们不会改变参数，所以定义为常量。
        #另一个方法：可以设置成trainable = False
        return tf.constant(self.data_dict[name][0], name = 'conv') #这里应该是w,b中的w
    
    def get_fc_weight(self, name):
        return tf.constant(self.data_dict[name][0], name = 'fc') #这里应该是w,b中的w
    
    def get_bias(self, name):
        return tf.constant(self.data_dict[name][1], name = 'bias') #这里应该是w,b中的b
    
    #创建卷积层，池化层，全连接层
    def conv_layer(self, x, name):
        """Builds convolution layer."""
        with tf.name_scope(name):
            #加上name_scope是命名规范：
            #1. 防止命名冲突
            #2. tensorboard打印名字更加清晰规范
            conv_w = self.get_conv_filter(name)
            conv_b = self.get_bias(name)
            
            #现在不再使用tf.layers.conv2d(),因为我们已经有了pre-trained的参数
            #现在使用更基础的api: tf.nn.conv2d()
            h = tf.nn.conv2d(x, conv_w, [1,1,1,1], padding = 'SAME') #x是input，[1,1,1,1]是strides步长，因为这里x是四维，所以我们输入四个数
            h = tf.nn.bias_add(h, conv_b)
            
            #激活函数
            h = tf.nn.relu(h)
            return h
        
    #也是使用tf.nn.max_pool()而不是tf.layers.max_pooling2d
    def pooling_layer(self, x, name):
        """Builds pooling layer."""
        return tf.nn.max_pool(x,
                                 ksize = [1,2,2,1], #因为是按照长和宽来池化，所以是中间两个维度是2，其余维度是1
                                 strides = [1,2,2,1], 
                                 padding = 'SAME', 
                                 name = name) 
    
    
    def fc_layer(self, x, name, activation = tf.nn.relu):
        """Builds fully-connected layer."""
        with tf.name_scope(name):
            fc_w = self.get_fc_weight(name)
            fc_b = self.get_bias(name)
            h = tf.matmul(x, fc_w) #让输入x与w进行操作
            h = tf.nn.bias_add(h, fc_b)
            if activation == None:
                return h
            else:
                return activation(h)
            
    
    #创建展平功能，展平后输入给全连接层：做的是reshape操作，我们需要知道reshape之后的size有多大
    #展平之后，需要的长宽厚的乘积
    def flatten_layer(self, x, name):
        """Builds flatten layer."""
        with tf.name_scope(name):
            #[batch_size, img_width, img_height, channel]
            x_shape = x.get_shape().as_list()
            dim = 1
            for d in x_shape[1:]:
                dim *= d
            x = tf.reshape(x, [-1, dim]) #这里的-1，是reshape推断出来的，也是我们的batch_size, 你也可以写成[x_shape[0], dim]
            return x
       
    #建立vgg
    #我们现在就要做图像的风格转换，需要的图片只有一个，所以第一个维度是1
    #vggnet的设置中，图像大小是224*224
    def build(self, x_rgb):
        """BUild VGG16 network structure.
        Args:
        - x_rgb: eg. [1, 224, 224, 3]
        """
        start_time = time.time()
        print("Building model...")
        
        #每个通道减去均值VGG_MEAN，先拆分通道
        #复习：tf.split() 之前用于：深度可分离卷积，数据增强
        r, g, b = tf.split(x_rgb, [1,1,1], axis = 3) #切分成三通道：[1,1,1]
        
        #去除均值后，需要合并。这里注意vggnet输入的通道顺序是BGR
        #意味着之前写的VGG_MEAN的三个数分别是 BGR 的均值
        x_bgr = tf.concat([b - VGG_MEAN[0], 
                           g - VGG_MEAN[1],
                           r - VGG_MEAN[2]],
                          axis = 3) #在第四个维度，channel上合并
        
        #预处理之后，判断一下我们的维度是 224*224*3
        assert x_bgr.get_shape().as_list()[1:] == [224,224,3]
        
        #构建前两个卷积层：
        #vgg16：
        #第一个结构(stage)：两个卷积层 -> 一个池化层
        #第二个结构：两个卷积层 -> 一个池化层
        #第3个结构：3个卷积层 -> 一个池化层
        #第4个结构：3个卷积层 -> 一个池化层
        #第5个结构：3个卷积层 -> 一个池化层
        #第6个结构：3个全连接层
        #2*2 + 3*3 + 3 = 4 + 9 + 3 = 16, 也就是vgg16 
        
        ##注意：self.conv_layer(xx,yy)第二个参数的名字必须是data_dict.keys()中的
        #dict_keys(['conv5_1', 'fc6', 'conv5_3', 'conv5_2', 'fc8', 'fc7', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'])
        #我们将每一个层，设置成了成员变量, eg. self.conv1_1, 可能会用其中的某一层计算风格损失或者内容损失，设置成成员变量我们以后可以方便使用
        self.conv1_1 = self.conv_layer(x_bgr, 'conv1_1')
        self.conv1_2 = self.conv_layer(self.conv1_1, 'conv1_2')
        self.pool1 = self.pooling_layer(self.conv1_2, 'pool1') #pool1因为不是data_dict里面预处理好的，所以我们可以随意命名, 例如pool1。并且可以不用将它设置成成员函数，只不过这里为了统一起见
        
        self.conv2_1 = self.conv_layer(self.pool1, 'conv2_1')
        self.conv2_2 = self.conv_layer(self.conv2_1, 'conv2_2')
        self.pool2 = self.pooling_layer(self.conv2_2, 'pool2')
        
        self.conv3_1 = self.conv_layer(self.pool2, 'conv3_1')
        self.conv3_2 = self.conv_layer(self.conv3_1, 'conv3_2')
        self.conv3_3 = self.conv_layer(self.conv3_2, 'conv3_3')
        self.pool3 = self.pooling_layer(self.conv3_3, 'pool3')
        
        self.conv4_1 = self.conv_layer(self.pool3, 'conv4_1')
        self.conv4_2 = self.conv_layer(self.conv4_1, 'conv4_2')
        self.conv4_3 = self.conv_layer(self.conv4_2, 'conv4_3')
        self.pool4 = self.pooling_layer(self.conv4_3, 'pool4')
        
        self.conv5_1 = self.conv_layer(self.pool4, 'conv5_1')
        self.conv5_2 = self.conv_layer(self.conv5_1, 'conv5_2')
        self.conv5_3 = self.conv_layer(self.conv5_2, 'conv5_3')
        self.pool5 = self.pooling_layer(self.conv5_3, 'pool5')
        
        '''
        #展开 -> 全连接
        self.flatten5 = self.flatten_layer(self.pool5, 'flatten')
        self.fc6 = self.fc_layer(self.flatten5, 'fc6')
        self.fc7 = self.fc_layer(self.fc6, 'fc7')
        
        #最后的fc8输出1k个值，给softmax()去计算概率分布
        #所以fc8不需要activation
        self.fc8 = self.fc_layer(self.fc7, 'fc8', activation = None)
        
        #计算softmax
        self.prob = tf.nn.softmax(self.fc8, name = 'prob')
        '''
        
        print("Building model finished: %4ds" % (time.time() - start_time))
        

### 测试我们的VGGNet 类是否成功：

In [5]:
#加载vgg16
vgg16_npy_path = '../../../other_datasets/vgg16.npy'
data_dict = np.load(vgg16_npy_path, encoding = 'latin1').item() #加item()是为了创建成字典

vgg16_for_result = VGGNet(data_dict)
content = tf.placeholder(tf.float32, shape = [1,224,224,3])
vgg16_for_result.build(content)

Building model...
Building model finished:    0s


### 开始风格转换

In [6]:
content_img_path = './others/resized_citi.JPG'
style_img_path = './others/resized_starry_night.jpg'

#训练次数
num_steps = 100

#学习率
learning_rate = 10

#风格系数，内容系数
lambda_c = 0 #内容
lambda_s = 1 #风格

#输出文件夹, 每一步都有新图像，将所有图像输入这个文件夹
output_dir = './005_style_rebuild_output_shallow'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [7]:
#定义随机的初始图片：
def initial_result(shape, mean, stddev):
    #用正态分布初始化
    initial = tf.truncated_normal(shape, mean = mean, stddev = stddev) #截断的正态分布
    return tf.Variable(initial)

#将风格，内容图像读取进来：
def read_img(img_name):
    #PIL里面的函数,读取img
    img = Image.open(img_name) 
    
    #变成numpy的矩阵
    np_img = np.array(img) #（224,224,3)
    
    #矩阵是 (224, 224, 3), 需要改变成4维：
    #因为只有一个样本，所以可以不用reshape，而是直接用[np_img]，将np_img包含在一个list [] 里面
    #如此就直接多了一个维度，变成(1, 224, 224, 3)
    np_img = np.asarray([np_img], dtype = np.int32)
    
    return np_img

#计算Gram矩阵，用于风格损失：
def gram_matrix(x):
    """Calculates gram matrix
    Args:
    - x: features extracted from VGG Net. shape: [1, width, height, channel]
    """
    batch, w, h, ch = x.get_shape().as_list()
    features = tf.reshape(x, [batch, h*w, ch]) #将w，h合成一个维度
    
    #针对channel，两两进行计算余弦相似度，得到gram matrix
    #1. 我们的features的第一个维度batch，永远都等于1
    #2. 我们的features的后两个维度，我们可以看成一个二维矩阵，假设叫A：行是h*w, 列是ch
    #3. 计算所有channel两两之间的相似度，我们可以看做将A矩阵中，抽取两个列进行点乘
    #4. 假设有k个channel，我们最后的gram matrix是k * k 矩阵，其中第i行，第j列代表第i个ch和第j个ch的相似度
    #5. 计算ch * ch的点乘，可以用 (ch * hw) * (hw * ch)进行点乘，也就是features的转置，点乘features
    #6. 矩阵乘法用tf.matmul(), 其中adjoin_a = True表示第一个参数要转置
    #7. 担心点乘后值过大，所以我们还会除以一个常数：也就是各个维度的乘积，例如224*224*3
    gram = tf.matmul(features, features, adjoint_a = True) \
            / tf.constant(ch * w * h, tf.float32)
    return gram

#初始化图像: mean是[0,255]的中间值
result = initial_result((1, 224, 224, 3), 127.5, 20)

#读取风格，内容图片
content_val = read_img(content_img_path)
style_val = read_img(style_img_path)

#用feed_dict塞入，所以我们需要先创建placeholder
content = tf.placeholder(tf.float32, shape = [1,224,224,3])
style = tf.placeholder(tf.float32, shape = [1,224,224,3])

#将三张图输入vggnet，然后提取特征
data_dict = np.load(vgg16_npy_path, encoding = 'latin1').item()

#创建三个vggnet，都是同样的参数
#给内容图像创建vggnet
vgg_for_content = VGGNet(data_dict)

#给风格图像创建vggnet
vgg_for_style = VGGNet(data_dict)

#给结果图像创建vggnet
vgg_for_result = VGGNet(data_dict)

#调用build函数，完成vggnet构建
#content和vgg_for_content进行关联
vgg_for_content.build(content)
vgg_for_style.build(style)
vgg_for_result.build(result)

#vggnet的每个层都可以进行特征提取
#content：越底层提取的特征越清晰
#以下是提取了5个特征，我们先注释掉后3个
content_features = [
    vgg_for_content.conv1_2,
    vgg_for_content.conv2_2,
    #vgg_for_content.conv3_3,
    #vgg_for_content.conv4_3,
    #vgg_for_content.conv5_3,
]

#给内容图像提取了哪些特征，就要对结果图像提取哪些特征
result_content_features = [
    vgg_for_result.conv1_2,
    vgg_for_result.conv2_2,
    #vgg_for_result.conv3_3,
    #vgg_for_result.conv4_3,
    #vgg_for_result.conv5_3,
]

#style: 越高层越好，所以注释掉底层
style_features = [
    vgg_for_style.conv1_2,
    vgg_for_style.conv2_2,
    #vgg_for_style.conv3_3,
    #vgg_for_style.conv4_3,
    #vgg_for_style.conv5_3,
]

#给风格图像的风格特征计算gram矩阵：
style_gram = [gram_matrix(feature) for feature in style_features]

#给风格图像提取了哪些特征，就要对结果图像提取哪些特征
result_style_features = [
    vgg_for_result.conv1_2,
    vgg_for_result.conv2_2,
    #vgg_for_result.conv3_3,
    #vgg_for_result.conv4_3,
    #vgg_for_result.conv5_3,
]

#给结果图像的风格特征计算gram矩阵：
result_style_gram = [gram_matrix(feature) for feature in result_style_features]

#计算内容损失 + 风格损失
#1. 内容损失：是每一层损失的加和
#zip: 将两个数组绑定在一起
#例如：[1,2], [3,4] 
#zip([1,2], [3,4]) -> [(1,3), (2,4)]
#两个list -> 一个list中两个pair：(1,3) 和 (2,4)
#想象：将两个list竖着放，然后横着拿出来
content_loss = tf.zeros(1, tf.float32) #一个数，是标量
for c, c_ in zip(content_features, result_content_features):
    #c, c_ = (content_features[0], result_content_features[0])
    #c, c_ = (content_features[1], result_content_features[1])
    #内容损失是提取的特征：c, c_的平方差，再求平均。
    #平均是长宽高所有维度的平均，因为shape = [1, width, height, channel], 所以axis = [1,2,3]
    content_loss += tf.reduce_mean((c - c_) ** 2, [1,2,3])
    
#2. 风格损失
#将某一层的特征提取出来，会得到feature_map
style_loss = tf.zeros(1, tf.float32)
for s, s_ in zip(style_gram, result_style_gram):
    #平方差损失函数，在1，2维度求均值。因为gram_matrix()已经将width和height降成1维。所以现在第零维是batch，第一维是w*h, 第二维度是ch
    style_loss += tf.reduce_mean((s - s_) ** 2, [1,2])

#loss加权
loss = content_loss * lambda_c + style_loss * lambda_s
    
#给损失函数计算梯度
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Building model...
Building model finished:    0s
Building model...
Building model finished:    0s
Building model...
Building model finished:    0s


### 图像风格转换的训练流程图

In [8]:
#定义初始化op
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op) #初始化变量，在图像风格转换中，变量只有一个：结果图像（对结果图像进行梯度下降）
    
    #开始训练：
    for step in range(num_steps):
        #最后一个_是因为要训练
        loss_value, content_loss_value, style_loss_value, _ \
            = sess.run([loss, content_loss, style_loss, train_op],
                        feed_dict = {
                            content: content_val,
                            style: style_val,
                        })
        print('step: %d, loss_value: %8.4f, content_loss: %8.4f, style_loss: %8.4f' \
             % (step+1, 
                loss_value[0],  #之所以是[0], 是因为我们是四维的，第一个维度只有一个元素，所以取[0]
                content_loss_value[0], 
                style_loss_value[0]))
        
        #获得每一步的结果图像，输出到dir中
        result_img_path = os.path.join(
            output_dir, 'result-%05d.jpg' % (step + 1))
        
        #将参数值，也就是结果图像的变量值，取出来
        #result是之前定义的随机噪声 result = initial_result((1, 224, 224, 3), 127.5, 20)
        result_val = result.eval(sess)[0] #因为第一个维度是只有一个元素，所以取出用[0]
        result_val = np.clip(result_val, 0, 255) #clip的作用：裁剪，将小于0的设成0，大于255的设置成255

        #之前是float，现在转换成int
        img_arr = np.asarray(result_val, np.uint8) 
        
        #用PIL转换成图片
        img = Image.fromarray(img_arr) #可以将numpy数组转换成图片
        img.save(result_img_path)
         

step: 1, loss_value: 1812998.5000, content_loss: 349758.5625, style_loss: 1812998.5000
step: 2, loss_value: 984988.2500, content_loss: 353371.4375, style_loss: 984988.2500
step: 3, loss_value: 504617.1250, content_loss: 414270.0312, style_loss: 504617.1250
step: 4, loss_value: 732134.5625, content_loss: 489737.4062, style_loss: 732134.5625
step: 5, loss_value: 485500.2812, content_loss: 498055.9375, style_loss: 485500.2812
step: 6, loss_value: 206284.4375, content_loss: 484390.9688, style_loss: 206284.4375
step: 7, loss_value: 133500.6719, content_loss: 473564.3125, style_loss: 133500.6719
step: 8, loss_value: 145045.9062, content_loss: 473493.8750, style_loss: 145045.9062
step: 9, loss_value: 134386.9062, content_loss: 483613.7500, style_loss: 134386.9062
step: 10, loss_value: 111831.5781, content_loss: 500374.2500, style_loss: 111831.5781
step: 11, loss_value: 122282.8750, content_loss: 516011.0625, style_loss: 122282.8750
step: 12, loss_value: 142127.5156, content_loss: 518321.0938,

step: 99, loss_value: 5962.0591, content_loss: 486252.9062, style_loss: 5962.0591
step: 100, loss_value: 5825.5918, content_loss: 486383.5938, style_loss: 5825.5918


### 为何之前设置这样的风格系数，内容系数
1. 首先我们的style loss的计算中，gram matrix需要除以一个比较大的数tf.constant(ch * w * h, tf.float32)，导致了style loss比较小
2. gram = tf.matmul(features, features, adjoint_a = True) \
            / tf.constant(ch * w * h, tf.float32)
3. 既然style loss小，content loss大，为了均衡两者，所以一个乘500，一个乘0.1
4. lambda_c = 0.1 #内容， lambda_s = 500 #风格
5. style loss: 1.7 * 500 = 850
6. content loss: 12997 * 0.1 = 1299

### 这个文件的泛化能力
1. 首先可以通过注释和取消注释来选取不同层的特征：

style_features = [
    #vgg_for_style.conv1_2,
    #vgg_for_style.conv2_2,
    #vgg_for_style.conv3_3,
    vgg_for_style.conv4_3,
    #vgg_for_style.conv5_3,
]

2. 可以只要重建图片，或者只重建风格：

lambda_c = 0 #只重建图片

lambda_s = 5000 #只重建风格

3. 可以为style loss 和 content loss的不同层添加系数：

for c, c_ in zip(content_features, result_content_features):

    content_loss += tf.reduce_mean((c - c_) ** 2, [1,2,3])
    
for s, s_ in zip(style_gram, result_style_gram):

    style_loss += tf.reduce_mean((s - s_) ** 2, [1,2])
    
例如第一层的c和c_可以乘系数0.1

content_loss += 0.1 * tf.reduce_mean((c - c_) ** 2, [1,2,3])

例如第二层的c和c_可以乘系数0.5

content_loss += 0.5 * tf.reduce_mean((c - c_) ** 2, [1,2,3])